In [6]:
import pandas as pd
import numpy as np

def get_highly_correlated_pairs(df: pd.DataFrame, threshold: float = 0.95):
    """
    データフレーム中のカラム間で、相関係数が指定した閾値以上の組み合わせを一覧で取得する関数
    
    Parameters
    ----------
    df : pd.DataFrame
        対象のデータフレーム（数値データのみ）
    threshold : float, default=0.95
        相関係数の閾値
    
    Returns
    -------
    correlated_pairs : pd.DataFrame
        カラムペアとその相関係数をまとめたDataFrame
    """
    # 相関行列を計算（絶対値を取る）
    corr_matrix = df.corr().abs()

    # 上三角行列のみを対象にして、重複を除く
    upper = corr_matrix.where(
        np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
    )

    # 閾値を超えるペアを抽出
    correlated_pairs = (
        upper.stack()
        .reset_index()
        .rename(columns={"level_0": "Column1", "level_1": "Column2", 0: "Correlation"})
    )
    correlated_pairs = correlated_pairs[correlated_pairs["Correlation"] >= threshold]

    # 相関係数の高い順にソート
    correlated_pairs = correlated_pairs.sort_values(by="Correlation", ascending=False).reset_index(drop=True)

    return correlated_pairs


In [7]:
# サンプルデータ
data = {
    "A": [1, 2, 3, 4, 5],
    "B": [2, 4, 6, 8, 10],
    "C": [5, 4, 3, 2, 1],
    "D": [1, 1, 1, 1, 1]
}

df = pd.DataFrame(data)

# 関数呼び出し
result = get_highly_correlated_pairs(df, threshold=0.95)
print(result)



  Column1 Column2  Correlation
0       A       B          1.0
1       A       C          1.0
2       B       C          1.0
